# Noiseless QCNN demo for Model3

This demo uses 4-qubit Quantum Convolutional Neural Network (QCNN) to see how pre-training the quantum embedding can be helpful for training a parameterized QML circuits for classfication tasks.

If you are interested in detailed analysis of pre-training on 8-qubit device, check out "/Result/earlystop 10 experiments/" folder.

If you are interested in the details about the QCNN used in this demo, check out https://arxiv.org/pdf/2108.00661.pdf.


In [1]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

# 0. Getting Started

Load the dataset

In [2]:
dev = qml.device('default.qubit', wires=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_reduction = False
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
def new_data(batch_size, X, Y):
    X1_new, X2_new, Y_new = [], [], []
    for i in range(batch_size):
        n, m = np.random.randint(len(X)), np.random.randint(len(X))
        X1_new.append(X[n])
        X2_new.append(X[m])
        if Y[n] == Y[m]:
            Y_new.append(1)
        else:
            Y_new.append(0)

    X1_new, X2_new, Y_new = torch.tensor(X1_new).to(torch.float32), torch.tensor(X2_new).to(torch.float32), torch.tensor(Y_new).to(torch.float32)
    if feature_reduction == False:
        X1_new = X1_new.permute(0, 3, 1, 2)
        X2_new = X2_new.permute(0, 3, 1, 2)
    return X1_new.to(device), X2_new.to(device), Y_new.to(device)

N_valid, N_test = 500, 1000
X1_new_valid, X2_new_valid, Y_new_valid = new_data(N_valid, X_test, Y_test)
X1_new_test, X2_new_test, Y_new_test = new_data(N_test, X_test, Y_test)

/var/folders/ym/ry24dhs911s7zbqfbhs5ns2m0000gn/T/ipykernel_9215/1083009800.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525498485/work/torch/csrc/utils/tensor_new.cpp:233.)
  X1_new, X2_new, Y_new = torch.tensor(X1_new).to(torch.float32), torch.tensor(X2_new).to(torch.float32), torch.tensor(Y_new).to(torch.float32)


# Part1: Pre-training the embedding

In [3]:
@qml.qnode(dev, interface="torch")
def Four_circuit3(inputs): 
    embedding.Four_QuantumEmbedding2(inputs[0:8])
    embedding.Four_QuantumEmbedding2_inverse(inputs[8:16])
    return qml.probs(wires=range(4))


class Four_Model3_Fidelity(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Layer1: 28 * 28 -> 14 * 14
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 8
        self.fc = torch.nn.Linear(7 * 7, 8, bias=True)

        self.qlayer3 = qml.qnn.TorchLayer(Four_circuit3, weight_shapes={})

    def forward(self, x1, x2):
        x1 = self.layer1(x1)
        x1 = self.layer2(x1)
        x1 = x1.view(-1, 7 * 7)
        x1 = self.fc(x1)

        x2 = self.layer1(x2)
        x2 = self.layer2(x2)
        x2 = x2.view(-1, 7 * 7)
        x2 = self.fc(x2)

        x = torch.concat([x1, x2], 1)
        x = self.qlayer3(x)
        return x[:,0]

class Four_Model3_HSinner(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.matrix_fn3 = qml.matrix(Four_circuit3)
        # Layer1: 28 * 28 -> 14 * 14
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 8
        self.fc = torch.nn.Linear(7 * 7, 8, bias=True)


    def forward(self, x1, x2):
        x1 = self.layer1(x1)
        x1 = self.layer2(x1)
        x1 = x1.view(-1, 7 * 7)
        x1 = self.fc(x1)

        x2 = self.layer1(x2)
        x2 = self.layer2(x2)
        x2 = x2.view(-1, 7 * 7)
        x2 = self.fc(x2)

        
        x = torch.concat([x1, x2], 1).to("cpu")
        x = [torch.real(torch.trace(self.matrix_fn3(a))) for a in x]
        x = torch.stack(x, dim=0).to(device)
        return x / 2**4

(Skip the code below, Already trained)

In [4]:
def train_models(model_name):
    train_loss = []
    if model_name == 'Model3_Fidelity':
        model = Four_Model3_Fidelity()
        PATH = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noiseless/Model 3/Four_Model3_Fidelity.pt'
    elif model_name == 'Model3_HSinner':
        model = Four_Model3_HSinner()
        PATH = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noiseless/Model 3/Four_Model3_HSinner.pt'
    model.train()

    loss_fn = torch.nn.MSELoss()
    opt = torch.optim.SGD(model.parameters(), lr=0.01)
    for it in range(1000):
        X1_batch, X2_batch, Y_batch = new_data(25, X_train, Y_train)
        X1_batch, X2_batch, Y_batch = X1_batch, X2_batch, Y_batch

        pred = model(X1_batch, X2_batch)
        pred, Y_batch = pred.to(torch.float32), Y_batch.to(torch.float32)
        loss = loss_fn(pred, Y_batch)
        train_loss.append(loss.item())

        opt.zero_grad()
        loss.backward()
        opt.step()

        if it % 200 == 0:
            print(f"Iterations: {it} Loss: {loss.item()}")
    
    torch.save(model.state_dict(), PATH)

#train_models('Model3_Fidelity')
train_models('Model3_HSinner')


Iterations: 0 Loss: 0.5999990105628967
Iterations: 200 Loss: 0.6799989938735962
Iterations: 400 Loss: 0.479999303817749
Iterations: 600 Loss: 0.5599992871284485
Iterations: 800 Loss: 0.39999961853027344


Calculate the distances of Test dataset with the pre-trained quantum embeddings. From the calculated trace distance gain the lower bound of the linear loss function (with respect to the test data).

In [7]:
X1_test, X0_test = [], []
for i in range(len(X_test)):
    if Y_test[i] == 1:
        X1_test.append(X_test[i])
    else:
        X0_test.append(X_test[i])
X1_test, X0_test = torch.tensor(X1_test).to(torch.float32), torch.tensor(X0_test).to(torch.float32)
X1_test, X0_test = X1_test.permute(0, 3, 1, 2), X0_test.permute(0, 3, 1, 2)

X1_train, X0_train = [], []
for i in range(len(X_train)):
    if Y_train[i] == 1:
        X1_train.append(X_train[i])
    else:
        X0_train.append(X_train[i])
X1_train, X0_train = torch.tensor(X1_train).to(torch.float32), torch.tensor(X0_train).to(torch.float32)
X1_train, X0_train = X1_train.permute(0, 3, 1, 2), X0_train.permute(0, 3, 1, 2)


@qml.qnode(dev, interface="torch")
def Four_Distance3(inputs): 
    embedding.Four_QuantumEmbedding2(inputs[0:8])
    return qml.density_matrix(wires=range(4))


class Distances3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Layer1: 28 * 28 -> 14 * 14
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 8
        self.fc = torch.nn.Linear(7 * 7, 8, bias=True)
        self.qlayer3_distance = qml.qnn.TorchLayer(Four_Distance3, weight_shapes={})

    def forward(self, x1, x0):
        x1 = self.layer1(x1)
        x1 = self.layer2(x1)
        x1 = x1.view(-1, 7 * 7)
        x1 = self.fc(x1)

        x0 = self.layer1(x0)
        x0 = self.layer2(x0)
        x0 = x0.view(-1, 7 * 7)
        x0 = self.fc(x0)

        rhos1 = self.qlayer3_distance(x1)
        rhos0 = self.qlayer3_distance(x0)
        
        rho1 = torch.sum(rhos1, dim=0) / len(x1)
        rho0 = torch.sum(rhos0, dim=0) / len(x0)
        rho_diff = rho1 - rho0
        eigvals = torch.linalg.eigvals(rho_diff)
        return 0.5 * torch.real(torch.sum(torch.abs(eigvals)))


PATH_Model3_Fidelity = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noiseless/Model 3/Four_Model3_Fidelity.pt'
PATH_Model3_HSinner = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noiseless/Model 3/Four_Model3_HSinner.pt'
Model3_Fidelity_Distance = Distances3().to(device)
Model3_Fidelity_Distance.load_state_dict(torch.load(PATH_Model3_Fidelity, map_location=device))
Model3_HSinner_Distance = Distances3().to(device)
Model3_HSinner_Distance.load_state_dict(torch.load(PATH_Model3_HSinner, map_location=device))

# Calculated from Model1 
Trace_before_traindata = 0.2734867334365845
Trace_before_testdata = 0.2723434865474701
LB_before_traindata = 0.5 * (1 - Trace_before_traindata)

print(f"Trace Distance (Training Data) Before: {Trace_before_traindata}")
print(f"Trace Distance (Test Data) Before: {Trace_before_testdata}")

# Distances After training with Model2_Fidelity
Trace_Fidelity_traindata = Model3_Fidelity_Distance(X1_train, X0_train)
Trace_Fidelity_testdata = Model3_Fidelity_Distance(X1_test, X0_test)
print(f"Trace Distance (Training Data) After Model3 Fidelity: {Trace_Fidelity_traindata}")
print(f"Trace Distance (Test Data) After Model3 Fidelity: {Trace_Fidelity_testdata}")

# Distances After training with Model2_HSinner
Trace_HSinner_traindata = Model3_HSinner_Distance(X1_train, X0_train)
Trace_HSinner_testdata = Model3_HSinner_Distance(X1_test, X0_test)
print(f"Trace Distance (Training Data) After Model3 HSinner: {Trace_HSinner_traindata}")
print(f"Trace Distance (Test Data) After Model3 HSinner: {Trace_HSinner_testdata}")

# Lower Bounds
LB_before_traindata = 0.5 * (1 - Trace_before_traindata)
LB_Fidelity_traindata = 0.5 * (1 - Trace_Fidelity_traindata.detach().numpy())
LB_HSinner_traindata = 0.5 * (1 - Trace_HSinner_traindata.detach().numpy())

Trace Distance (Training Data) Before: 0.2734867334365845
Trace Distance (Test Data) Before: 0.2723434865474701


/Users/tak/miniforge3/envs/QC/lib/python3.10/site-packages/pennylane/qnn/torch.py:328: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525498485/work/aten/src/ATen/native/Copy.cpp:250.)
  return self.qnode(**kwargs).type(x.dtype)


Trace Distance (Training Data) After Model3 Fidelity: 0.914618194103241
Trace Distance (Test Data) After Model3 Fidelity: 0.9234920740127563
Trace Distance (Training Data) After Model3 HSinner: 1.1824748071376234e-05
Trace Distance (Test Data) After Model3 HSinner: 1.2403033906593919e-05


# Part2: Training QCNN with/without Pre-trained embedding

Relabel the datasets

In [27]:
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
X_train, X_test = X_train.permute(0, 3, 1, 2), X_test.permute(0, 3, 1, 2)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [13]:
class x_transform3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 8
        self.fc = torch.nn.Linear(7 * 7, 8, bias=True)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 7 * 7)
        x = self.fc(x)
        return x.detach().numpy()

model = x_transform3().to(device)

<All keys matched successfully>

Tunable Hyperparameters

In [ ]:
steps = 1000
learning_rate = 0.01
batch_size = 128
ansatz = 'SU4'

In [16]:
def statepreparation(x, Trained):
    if Trained == 'Model3_Fidelity':
        model.load_state_dict(torch.load(PATH_Model3_Fidelity, map_location=device))
        x = model(torch.tensor(x))
    elif Trained == 'Model3_HSinner':
        model.load_state_dict(torch.load(PATH_Model3_HSinner, map_location=device))
        x = model(torch.tensor(x))
    embedding.Four_QuantumEmbedding2(x)


@qml.qnode(dev)
def QCNN_classifier(params, x, Trained):
    statepreparation(x, Trained)
    embedding.QCNN_four(params, ansatz)
    return qml.expval(qml.PauliZ(2))


def Linear_Loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss += 0.5 * (1 - l * p)
    return loss / len(labels)


def cost(weights, X_batch, Y_batch, Trained):
    preds = [QCNN_classifier(weights, x, Trained) for x in X_batch]
    return Linear_Loss(Y_batch, preds)


def circuit_training(X_train, Y_train, Trained):

    if ansatz == 'SU4':
        num_weights = 30
    elif ansatz == 'TTN':
        num_weights = 2

    weights = np.random.random(num_weights, requires_grad = True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history = []
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        weights, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, Trained),
                                                     weights)
        loss_history.append(cost_new)
        if it % 200 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return loss_history, weights

In [ ]:
Loss_histories_Model3_Fidelity, weights_Model3_Fidelity, Loss_histories_Model3_HSinner, weights_Model3_HSinner = [], [], [], []
for i in range(5):
    loss_Model3_Fidelity, weight_Model3_Fidelity = circuit_training(X_train, Y_train, 'Model3_Fidelity')
    loss_Model3_HSinner, weight_Model3_HSinner = circuit_training(X_train, Y_train, 'Model3_HSinner')


    Loss_histories_Model3_Fidelity.append(loss_Model3_Fidelity)
    weights_Model3_Fidelity.append(weight_Model3_Fidelity)

    Loss_histories_Model3_HSinner.append(loss_Model3_HSinner)
    weights_Model3_HSinner.append(weight_Model3_HSinner)


Loss_histories_Model3_Fidelity, Loss_histories_Model3_HSinner =  np.array(Loss_histories_Model3_Fidelity), np.array(Loss_histories_Model3_HSinner)

Model3_Fidelity_mean, Model3_Fidelity_std = Loss_histories_Model3_Fidelity.mean(axis=0), Loss_histories_Model3_Fidelity.std(axis=0)
Model3_HSinner_mean, Model3_HSinner_std = Loss_histories_Model3_HSinner.mean(axis=0), Loss_histories_Model3_HSinner.std(axis=0)

In [ ]:
from numpy import genfromtxt

Loss_histories_not_trained = []

for i in range(5):
    Loss_histories_not_trained_PATH = f'/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noiseless/Model 1/Loss_histories_not_trained/Loss_histories_not_trained{i + 1}.txt'
    textfile = open(Loss_histories_not_trained_PATH, 'r')
    data = np.array([])
    for line in textfile:
        a = np.array([float(v) for v in line.strip().split(" ")])
        data = np.append(data, a)
    Loss_histories_not_trained.append(data.tolist())


Loss_histories_not_trained = np.array(Loss_histories_not_trained)
Not_trained_mean, Not_trained_std = Loss_histories_not_trained.mean(axis=0), Loss_histories_not_trained.std(axis=0)

In [ ]:
import seaborn as sns

plt.rcParams['figure.figsize'] = [10, 5]
fig, ax = plt.subplots()
clrs = sns.color_palette("husl", 3)
with sns.axes_style("darkgrid"):
    ax.plot(range(len(Not_trained_mean)), Not_trained_mean, label="No Pre-training", c=clrs[0])
    ax.fill_between(range(len(Not_trained_mean)), Not_trained_mean-Not_trained_std, Not_trained_mean+Not_trained_std, alpha=0.3,facecolor=clrs[0])

    ax.plot(range(len(Model3_Fidelity_mean)), Model3_Fidelity_mean, label="Model3 Fidelity Pre-training", c=clrs[1])
    ax.fill_between(range(len(Model3_Fidelity_mean)), Model3_Fidelity_mean-Model3_Fidelity_std, Model3_Fidelity_mean+Model3_Fidelity_std, alpha=0.3,facecolor=clrs[1])

    ax.plot(range(len(Model3_HSinner_mean)), Model3_HSinner_mean, label="Model3 HSinner Pre-training", c=clrs[2])
    ax.fill_between(range(len(Model3_HSinner_mean)), Model3_HSinner_mean-Model3_HSinner_std, Model3_HSinner_mean+Model3_HSinner_std, alpha=0.3,facecolor=clrs[2])

    ax.plot(range(1000), np.ones(1000) * LB_before_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound without Pre-training", c=clrs[0])
    ax.plot(range(1000), np.ones(1000) * LB_Fidelity_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound with Model3 Fidelity", c=clrs[1])
    ax.plot(range(1000), np.ones(1000) * LB_HSinner_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound with Model3 HSinner", c=clrs[2])


ax.set_xlabel("Iteration")
ax.set_ylabel("Loss")
ax.set_title("Model3 QCNN Loss History")
ax.legend()

Save the Loss Histories and Trained weights

In [ ]:
f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noiseless/Model 3/Loss_histories_and_weights.txt', 'w')

for i in range(5):
    f.write(f'Loss History Model3 Fidelity {i + 1}:')
    f.write('\n')
    f.write(str(Loss_histories_Model3_Fidelity[i]))
    f.write('\n')
for i in range(5):
    f.write(f'Weights Model3 Fidelity {i + 1}:')
    f.write('\n')
    f.write(str(weights_Model3_Fidelity[i]))
    
for i in range(5):
    f.write(f'Loss History Model3 HSinner {i + 1}:')
    f.write('\n')
    f.write(str(Loss_histories_Model3_HSinner[i]))
    f.write('\n')
for i in range(5):
    f.write(f'Weights Model3 HSinner {i + 1}:')
    f.write('\n')
    f.write(str(weights_Model3_HSinner[i]))
f.close()

Check the accuracies of QCNN classifiers

In [ ]:
def accuracy_test(predictions, labels):
    acc = 0
    for l, p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)


#accuracies_not_trained = []
accuracies_Model3_Fidelity, accuracies_Model3_HSinner = [], []

for i in range(5):
    #prediction_not_trained = [QCNN_classifier(weights_not_trained[i], x, Trained=False) for x in X_test]
    prediction_Model3_Fidelity = [QCNN_classifier(weights_Model3_Fidelity[i], x, Trained='Model3_Fidelity') for x in X_test]
    prediction_Model3_HSinner = [QCNN_classifier(weights_Model3_HSinner[i], x, Trained='Model3_HSinner') for x in X_test]
    
    #accuracy_not_trained = accuracy_test(prediction_not_trained, Y_test)
    accuracy_Model3_Fidelity = accuracy_test(prediction_Model3_Fidelity, Y_test)
    accuracy_Model3_HSinner = accuracy_test(prediction_Model3_HSinner, Y_test)

    #accuracies_not_trained.append(accuracy_not_trained)
    accuracies_Model3_Fidelity.append(accuracy_Model3_Fidelity)
    accuracies_Model3_HSinner.append(accuracy_Model3_HSinner)

#accuracies_not_trained = np.array(accuracies_not_trained),
accuracies_Model3_Fidelity, accuracies_Model3_HSinner = np.array(accuracies_Model3_Fidelity), np.array(accuracies_Model3_HSinner)

print(f" Accuracy without pre-training: 0.6604255319148936 ± 0.01512233847372428")
print(f" Accuracy after pre-training with Model2_Fidelity: {accuracies_Model3_Fidelity.mean()} ± {accuracies_Model3_Fidelity.std()}")
print(f" Accuracy after pre-training with Model2_HSinner: {accuracies_Model3_HSinner.mean()} ± {accuracies_Model3_HSinner.std()}")